### **Import the Necessary Libraries**

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

#### **Load the Face Detector Model**

In [2]:
# Load the serialized face detector model
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

### **Load the Pre-trained CNN Model**

In [3]:
# Load the model for face mask detection
maskClassifier = load_model("cnn-model.h5")

# Define model constants 
img_size = 224
channels = 3

### **Define the functions**

The `detect_face` function is used for detecting the faces captured by the webcam. This function returns a tuple containing a list of extracted faces and a list of the faces' positions.

In [4]:
def detect_face(frame):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (img_size, img_size), (104.0, 177.0, 123.0))

    # Pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # Initialize the list of faces and their corresponding locations
    faces = []
    locs = []

    # Loop over the detections
    for i in range(0, detections.shape[2]):
        # Extract the confidence (i.e., probability) associated with the detection
        confidence = detections[0, 0, i, 2]
        
        # Filter out weak detections by setting a minimum confidence
        if confidence > 0.5:
            # Compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # Extract the face's region of interest (ROI)
            faceROI = frame[startY:endY, startX:endX]
            # Convert the extracted face from BGR to RGB channel
            faceROI = cv2.cvtColor(faceROI, cv2.COLOR_BGR2RGB)

            # Ddd the face and bounding boxes to their respective lists
            faces.append(faceROI)
            locs.append((startX, startY, endX, endY))
            
    return (faces, locs)
          

The `predict` function is used to make predictions using the CNN model. This function first preprocesses the image input to fit the required image size. Afterwards, using the imported model, the predictions are made.

In [5]:
def predict(face):
    # Preprocess the image
    face = cv2.resize(face, (img_size, img_size))
    face = img_to_array(face)
    face = face.reshape(1, img_size, img_size, channels)

    # Make predictions
    prediction = maskClassifier(face)
    return prediction[0].numpy()


The `add_ARFilter` function is used to add the Augmented Reality (AR) Filter over the faces of the people captured by the webcam. The filter changes color depending on the prediction and a label is added.

In [6]:
def add_ARFilter(frame, faces, locs):
    preds = []

    # Only make predictions when there are faces detected
    if len(faces) > 0:
        for face in faces:
            preds.append(predict(face))
    
    # Only add filter when there are faces detected and predictions made
    if len(locs) > 0 and len(preds) > 0:
        for (box, pred) in zip(locs, preds):
            # Unpack the bounding box and predictions
            (startX, startY, endX, endY) = box
            (withmask, withoutmask) = pred

            # Define the label and color of the filter
            (label, color) = (
                (f"Mask: {np.round(withmask * 100.00, 2)}%", (0, 255, 0))
                if withmask > withoutmask
                else (f"No Mask: {np.round(withoutmask * 100.00, 2)}%", (0, 0, 255), )
            )

            # Add filter (text label and bounding box) using OpenCV
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

### **Open the Webcam**

In [8]:
# Define a video capture object to read input from the webcam
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if cap.isOpened() == False:
    print("Error opening webcam")

# Set the size of the video window
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

True

### **Detect and Classify Face Mask Use**

In [9]:
# Start reading the webcam stream and detect face masks in the video
while True:
    stream_ok, frame = cap.read()
    # Check if the webcam stream is available
    if not stream_ok:
        print("Error reading webcam")
        break

    frame = cv2.flip(frame, 1)

    # Detect faces and facemasks in the video frame using detect_face function
    (faces, locs) = detect_face(frame)

    # Add AR filter to the video frame based on the predictions
    add_ARFilter(frame, faces, locs)
    
    # Displaying the current video frame
    cv2.imshow("Video", frame)

    # Press 'q' to exit the video stream
    if cv2.waitKey(1) & 0xFF == ord("q"):
        print("Webcam has been closed")
        break

# Video Stream cleanup and capture release
cap.release()  
cv2.destroyAllWindows()


Webcam has been closed
